In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess

In [3]:
train_results_dir = os.getcwd() + "/../002-Training-models/train_results/chosen_models"
!ls {train_results_dir}

model_yolov8_tf_yolov8l_imgsz_640_epochs_50_batch_16_dataset_v2_loss_SGD_lr_01
model_yolov8_tf_yolov8m_imgsz_640_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01
model_yolov8_tf_yolov8m_imgsz_640_epochs_50_batch_16_dataset_v2_loss_SGD_lr_01
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01
model_yolov8_tf_yolov8s_imgsz_800_epochs_60_batch_16_dataset_v2_loss_SGD_lr_01


In [4]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../datasets/"

# Evaluate

In [5]:
def evaluate_models(train_results_dir, output_file, dataset_dir, task="val", dataset_sufix="_original", dataset_version="v1"):
    %cd {eval_results_dir}
    total = len(os.listdir(train_results_dir))
    counter = 0
    for folder_name in sorted(os.listdir(train_results_dir)):
        counter += 1
        if not folder_name.startswith("model"):
            continue

        folder_path = os.path.join(train_results_dir, folder_name, "weights")
        print(f"Evaluatig models on {task}: {counter}/{total}")

        # Split the folder name into variables
        variables = folder_name.split('_')

        # Extract the required variables
        model = variables[1]
        tf = variables[3]
        imgsz = variables[5]
        epochs = variables[7]
        batch = variables[9]
        dataset = variables[11]
        loss = variables[13]
        lr = variables[15]

        print(folder_name)
        print("Model:", model)
        print("TF:", tf)
        print("Imgsz:", imgsz)
        print("Epochs:", epochs)
        print("Batch:", batch)
        print("Dataset:", dataset)
        print("Executing command...")
        print()

        dataset_location = dataset_dir + model + dataset_sufix

        command = None

        if model == "yolov8" and dataset == dataset_version:
            best_pt_path = os.path.join(folder_path, "best.pt")
            if os.path.exists(best_pt_path):
                %cd yolov8
                command = f"yolo val model={best_pt_path} data={dataset_location}/data.yaml split={task} name={folder_name}"
                print(command)

        if command:
            # Run the command using subprocess.Popen and capture the output
            with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
                # Open the output file for writing
                with open(f"{eval_results_dir}/{output_file}", "a") as file:
                    file.write(folder_name)
                    file.write(f"\nModel: {model}")
                    file.write(f"\nTF: {tf}")
                    file.write(f"\nImgsz: {imgsz}")
                    file.write(f"\nEpochs: {epochs}")
                    file.write(f"\nBatch: {batch}")
                    file.write(f"\nDataset: {dataset}")
                    file.write(f"\nLoss: {loss}")
                    file.write(f"\nLr0: {lr}\n")
                    # Read and display each line of the output as it becomes available
                    for line in process.stdout:
                        print(line, end="")
                        file.write(line)
            %cd ..

        from IPython import display
        display.clear_output()


    print("Executing commands...", "DONE!")
    print()

In [6]:
def process_file(file_path, csv_output_file):
    results = list()
    curr_res = dict()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Model:"):
            curr_res["model"] = line.split()[1]
        elif line.startswith("TF:"):
            curr_res["transfer_learning"] = line.split()[1]
        elif line.startswith("Imgsz:"):
            curr_res["imgsz"] = line.split()[1]
        elif line.startswith("Epochs:"):
            curr_res["epochs"] = line.split()[1]
        elif line.startswith("Batch:"):
            curr_res["batch"] = line.split()[1]
        elif line.startswith("Dataset:"):
            curr_res["dataset"] = line.split()[1]
        elif line.startswith("Loss:"):
            curr_res["optimizer"] = line.split()[1]
        elif line.startswith("Lr0:"):
            try:
                curr_res["lr0"] = float("0."+line.split()[1])
            except:
                curr_res["lr0"] = float(line.split()[1])
            
        if line.strip(" ").startswith("all"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["all_P"] = P
            curr_res["all_R"] = R
            curr_res["all_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["all_mAP@.5"] = mAP5
            curr_res["all_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("knife"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["knife_P"] = P
            curr_res["knife_R"] = R
            curr_res["knife_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["knife_mAP@.5"] = mAP5
            curr_res["knife_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("pistol") or line.strip(" ").startswith("gun"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["pistol_P"] = P
            curr_res["pistol_R"] = R
            curr_res["pistol_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["pistol_mAP@.5"] = mAP5
            curr_res["pistol_mAP@.5:.95"] = mAP95
        
        elif line.strip(" ").startswith("Speed:"):
            ms = 0
            for i in [1,3,5,7]:
                ms += float(line.replace("ms","").split(" ")[i])
            curr_res["FPS"] = 1000/ms

        elif line.strip(" ").startswith("Results saved to"):
            model_key = line.split("/")[-1].replace('\x1b[0m', '').replace('\n', '')
            model_info = model_key.split("_")

            curr_res["model_key"] = model_key

            results.append(curr_res)
            curr_res = dict()

    df = pd.DataFrame(results)
    df.set_index('model_key', inplace=True)
    df.to_csv(csv_output_file)

## Evaluate

In [20]:
def run_evaluation(eval_results_dir, train_results_dir, dataset_dir, task, dataset_sufix="_original", dataset_version="v1", ds_name=""):
    
    output_file_name = f"eval_trained_on_dataset_{dataset_version}_task_{task}_ds_{ds_name}"
    
    output_file = f"logs/{output_file_name}.txt"
    csv_output_file = f'{eval_results_dir}/output_csvs/{output_file_name}.csv'
    print(output_file)

    evaluate_models(train_results_dir, output_file, dataset_dir, task, dataset_sufix, dataset_version)
    process_file(os.path.join(eval_results_dir, output_file), csv_output_file)

In [21]:
dirs = [eval_results_dir, train_results_dir, dataset_dir]

In [22]:
dirs

['/home/flor/Desktop/git_repos/YOLO-OpenVINO-TVM-GStreamer/003-Evaluating-models',
 '/home/flor/Desktop/git_repos/YOLO-OpenVINO-TVM-GStreamer/003-Evaluating-models/../002-Training-models/train_results/chosen_models',
 '/home/flor/Desktop/git_repos/YOLO-OpenVINO-TVM-GStreamer/003-Evaluating-models/../datasets/']

In [23]:
run_evaluation(dirs[0], dirs[1], dirs[2], "test", dataset_sufix="_rc_no_empty", dataset_version="v2", ds_name="rc")

Executing commands... DONE!

